In [104]:
import json

with open("C:/Users/Administrator/Desktop/spec/vergil_wrangle/fall2020/fall2020(jan2).json", "r") as read_file:
    vergil = json.load(read_file)
    read_file.close() 
print(len(vergil))

with open("C:/Users/Administrator/Desktop/spec/vergil_wrangle/spring2021/spring2021(jan2).json", "r") as read_file:
    vergil = json.load(read_file)
    read_file.close()   
print(len(vergil))

with open("C:/Users/Administrator/Desktop/spec/vergil_wrangle/spring2020/spring2020(july20).json", "r") as read_file:
    vergil = json.load(read_file)
    read_file.close()
print(len(vergil))

with open("C:/Users/Administrator/Desktop/spec/vergil_wrangle/fall2020/fall2020ugradData.json", "r") as read_file:
    vergil = json.load(read_file)
    read_file.close() 
print(len(vergil))

with open("C:/Users/Administrator/Desktop/spec/vergil_wrangle/spring2021/spring2021ugradData.json", "r") as read_file:
    vergil = json.load(read_file)
    read_file.close()   
print(len(vergil))

with open("C:/Users/Administrator/Desktop/spec/vergil_wrangle/spring2020/spring2020ugradData.json", "r") as read_file:
    vergil = json.load(read_file)
    read_file.close()
print(len(vergil))

4383
3902
4046
1600
1587
1765


## Reformat original json files
### Missing data:
- spring 2020:
    - ugrad course: 2060
    - ugrad courses with time info: 1765
- fall 2020
    - ugrad course: 1918
    - ugrad courses with time info: 1600
- spring 2021
    - ugrad course: 1860
    - ugrad courses with time info: 1587

In [116]:
import json

with open("C:/Users/Administrator/Desktop/spec/vergil_wrangle/spring2020/spring2020(july20).json", "r") as read_file:
    vergil = json.load(read_file)
    read_file.close()

#put all 0000-4000 level courses in 'undergrad' array
undergrad = []
for course in vergil:
    courseId = course["course"]["course_identifier"]
    
    if len(courseId) == 9 or len(courseId) == 10:
        if courseId[-4].isdigit():
            if int(courseId[-4]) <= 4:
                #print(courseId)
                undergrad.append(course["course"])
                
#print(len(undergrad)) 
#print(undergrad[2:3])

#extract each course section's time and max enrollment
alltimeDicts = {}
types = []
for course in undergrad:
    courseId = course["course_identifier"]
    
    sections = course["classes"]["class"]
    times = []
    info = {}
    
    
    for section in sections:
        timesListed = section["days_times"]
        size = section["enrollment"]["max"]
        classtype = section["type"]["type_code"]
        classtypename = section["type"]["name"]
        
        #print(classtype, classtypename)
        
        for day in timesListed:
            startT = day["mil_time_from"]
            endT = day["mil_time_to"]
            time = [startT, endT]
            if startT and endT:
                times.append(time)
                if classtypename not in types:
                    types.append(classtypename)
    
    
    if times:
        info.update({"size": size})
        info.update({"times": times})
        info.update({"type": classtypename})
        
    if info:
        alltimeDicts.update({courseId: info})

#for type in types:
    #print(type)

#print(len(alltimeDicts))
#print(alltimeDicts)

#write to json

#with open("spring2021ugradData.json","w") as write:
    #json.dump(alltimeDicts, write, indent = 4)

## Convert time zones to .json

In [177]:
from datetime import datetime, timezone
import pytz
from parse import *
import json

with open("C:/Users/Administrator/Desktop/spec/vergil_wrangle/spring2021/spring2021ugradData.json", "r") as read_file:
    ugrad = json.load(read_file)
    read_file.close()
    
#for tz in pytz.all_timezones:
    #print(tz)

#set standard date as 01-01-2020 
#not during daylight savings, so 23 hrs between nyc and beijing instead of 24
standard = "01-01-2020 "

fmt = "%H:%M"

#timezone variables
ny = pytz.timezone('America/New_York')
bj = pytz.timezone('Asia/Shanghai')
la = pytz.timezone('America/Los_Angeles')
london =pytz.timezone('Europe/London')
moscow =pytz.timezone('Europe/Moscow')
vancouver =pytz.timezone('Etc/GMT+8')
mexico =pytz.timezone('Mexico/General')
sao_paolo =pytz.timezone('America/Sao_Paulo')
athens =pytz.timezone('Europe/Athens')
baghdad =pytz.timezone('Asia/Baghdad')
bangkok =pytz.timezone('Asia/Bangkok')
syndey =pytz.timezone('Australia/Sydney')

def convert(targetZone, timeStr):
    dt = datetime.strptime(timeStr, "%d-%m-%Y %H:%M:%S")
    dt = ny.localize(dt)
    dtconverted = dt.astimezone(targetZone)
    return dtconverted.strftime(fmt)

arr = ugrad
with open('spring2021_syndey_endTimes.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    for course in arr:
        for time in arr[course]["times"]:
            start = time[0]
            start = standard + start + ":00"
            time[0] = convert(bangkok, start)

            end = time[1]
            end = standard + end + ":00"
            time[1] = convert(bangkok, end)
            writer.writerow([time[1]])

'''
for course in arr:
    for time in arr[course]["times"]:
        start = time[0]
        start = standard + start + ":00"
        time[0] = convert(la, start)

        end = time[1]
        end = standard + end + ":00"
        time[1] = convert(la, end)
        
with open("spring2021ugrad_la.json","w") as write:
    json.dump(arr, write, indent = 4)

with open("spring2021ugrad_syndey.json","w") as write:
    json.dump(arr, write, indent = 4)
    
#print(bjArr)
#print(bjStartArr)

#with open("spring2020ugrad_bj.json","w") as write:
    #json.dump(bjArr, write, indent = 4)
'''

'\nfor course in arr:\n    for time in arr[course]["times"]:\n        start = time[0]\n        start = standard + start + ":00"\n        time[0] = convert(la, start)\n\n        end = time[1]\n        end = standard + end + ":00"\n        time[1] = convert(la, end)\n        \nwith open("spring2021ugrad_la.json","w") as write:\n    json.dump(arr, write, indent = 4)\n\nwith open("spring2021ugrad_syndey.json","w") as write:\n    json.dump(arr, write, indent = 4)\n    \n#print(bjArr)\n#print(bjStartArr)\n\n#with open("spring2020ugrad_bj.json","w") as write:\n    #json.dump(bjArr, write, indent = 4)\n'

## Check class sizes

In [99]:
import json
import statistics as stats

with open("C:/Users/Administrator/Desktop/spec/vergil_wrangle/fall2020/fall2020ugradData.json", "r") as read_file:
    courses = json.load(read_file)
    read_file.close()
    
size = []    
nullCount = 0
for course in courses:
    if not courses[course]["size"]:
        #print(course)
        nullCount+=1
    if not courses[course]["times"][0]:
        print(course)
    else:
        if courses[course]["type"] == "Studio":
            #print(course , courses[course]["size"])
            size.append(int(courses[course]["size"]))
        
#print(stats.mean(size))

In [129]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.collections import PolyCollection
from collections import Counter
import csv


with open("C:/Users/Administrator/Desktop/spec/vergil_wrangle/spring2021/spring2021ugrad_bj.json", "r") as read_file:
    courses = json.load(read_file)
    read_file.close()

endTimes = []
for course in courses:
    for time in courses[course]["times"]:
        endTime = time[1]
        print(course,endTime)
        endTimes.append(endTime)

with open('spring2021_bj_endTimes.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    for course in courses:
        for time in courses[course]["times"]:
            endTime = time[1]
            writer.writerow(endTime)
 
print(len(endTimes))
freq = Counter(endTimes)

for k,v in freq.items():
    print(k,v)

'''
with open('spring2020_freq.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    for k,v in freq.items():
        writer.writerow([k,v])
'''

SCNCC1500 00:50
BUSIW3701 00:25
BUSIW3021 04:55
BUSIW3013 04:55
GERMBC3057 00:25
GERMBC3022 07:00
GERMV3002 06:25
GERMW2102 09:00
DNCEBC2138 01:55
DNCEBC1820 00:25
DNCEBC1333 04:55
DNCEBC1331 23:15
DNCEBC1250 06:25
DNCEBC1248 03:25
DNCEBC1138 03:25
DNCEBC3602 06:00
DNCEBC3602 17:00
DNCEBC3593 01:00
DNCEBC3576 07:00
DNCEBC3336 03:30
DNCEBC3333 06:25
DNCEBC3250 03:00
DNCEBC3249 04:55
DNCEBC3141 04:30
DNCEBC3139 22:55
DNCEBC3139 09:55
DNCEBC3002 03:25
DNCEBC1136 06:25
DNCEBC1136 17:25
DNCEBC2565 04:55
DNCEBC2563 03:25
DNCEBC2455 06:25
DNCEBC2452 00:25
DNCEBC2335 04:55
DNCEBC2333 01:55
DNCEBC2255 06:00
DNCEBC2253 01:55
DNCEBC2252 00:25
DNCEBC2140 06:25
ANATBC2573 04:55
FYWBBC1114 03:25
FYWBBC1113 04:55
FYWBBC1113 15:55
FYWBBC1112 01:55
FYWBBC1112 12:55
FYWBBC1111 03:25
FYWBBC1106 01:55
FYWBBC1105 04:55
FYWBBC1105 15:55
FYWBBC1104 03:25
FYWBBC1104 14:25
FYWBBC1102 03:25
FYWBBC1102 14:25
FYWBBC1119 01:55
FYWBBC1118 00:25
FYWBBC1117 04:55
FYWBBC1117 15:55
FYWBBC1116 03:25
FYWBBC1115 01:55
FYW

"\nwith open('spring2020_freq.csv', mode='w', newline='') as file:\n    writer = csv.writer(file)\n    for k,v in freq.items():\n        writer.writerow([k,v])\n"

## Convert course end times in a single .csv

In [193]:
import csv
from datetime import datetime, timezone
import pytz
from parse import *

#set standard date as 01-01-2020 
#not during daylight savings, so 23 hrs between nyc and beijing instead of 24
standard = "01-01-2020 "

fmt = "%H:%M"

#timezone variables
ny = pytz.timezone('America/New_York')
bj = pytz.timezone('Asia/Shanghai')
la = pytz.timezone('America/Los_Angeles')
london =pytz.timezone('Europe/London')
moscow =pytz.timezone('Europe/Moscow')
vancouver =pytz.timezone('Etc/GMT+8')
mexico =pytz.timezone('Mexico/General')
sao_paolo =pytz.timezone('America/Sao_Paulo')
athens =pytz.timezone('Europe/Athens')
baghdad =pytz.timezone('Asia/Baghdad')
bangkok =pytz.timezone('Asia/Bangkok')
syndey =pytz.timezone('Australia/Sydney')


with open("C:/Users/Administrator/Desktop/spec/vergil_wrangle/fall2020/fall2020_endTimes.csv", newline='') as file:
    reader = csv.reader(file)
    
    with open("C:/Users/Administrator/Desktop/spec/vergil_wrangle/fall2020_endTimes.csv", 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["nyc", "bj", "la", "london", "moscow", "vancouver", "mexico", "sao_paolo", "athens", "baghdad", "bangkok", "syndey"])
        for row in reader:
            nyc = standard + row[0] + ":00"

            Tnyc = convert(ny, nyc)
            Tbj = convert(bj, nyc)
            Tla = convert(la, nyc)
            Tlondon = convert(london, nyc)
            Tmoscow = convert(moscow, nyc)
            Tvancouver = convert(vancouver, nyc)
            Tmexico = convert(mexico, nyc)
            Tsao_paolo = convert(sao_paolo, nyc)
            Tathens = convert(athens, nyc)
            Tbaghdad = convert(baghdad, nyc)
            Tbangkok = convert(bangkok, nyc)
            Tsyndey = convert(syndey, nyc)

            writer.writerow([Tnyc, Tbj, Tla, Tlondon, Tmoscow, Tvancouver, Tmexico, Tsao_paolo, Tathens, Tbaghdad, Tbangkok, Tsyndey ])


def convert(targetZone, timeStr):
    dt = datetime.strptime(timeStr, "%d-%m-%Y %H:%M:%S")
    dt = ny.localize(dt)
    dtconverted = dt.astimezone(targetZone)
    return dtconverted.strftime(fmt)